In [1]:
import numpy as np
import pandas as pd
import wandb
from wd.utils.utilities import mlflow_linearize

In [2]:
from IPython.core.display_functions import display


def get_runs(run_filters):
    # Project is specified by <entity/project-name>
    runs = api.runs("pasqualedem/weeddetection", filters=run_filters)

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
              if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
    df = pd.DataFrame([{**mlflow_linearize(dc), **mlflow_linearize(ds)} for dc, ds in zip(config_list, summary_list)])
    return df

def remove_hyp(df):
    to_delete = filter(lambda x: x.startswith('hyper_params'), df.columns)
    display(list(to_delete))
    for l in to_delete:
        df.drop(l, inplace=True)

def rename_values(df: pd.DataFrame, vrenamings: dict):
    for column, values in vrenamings.items():
        if column in df.columns:
            df[column] = df[column].replace(values)

In [3]:
api = wandb.Api()

In [4]:
renamings = {
    'in_params.dataset.channels': 'Channels',
    'in_params.train_params.optimizer_params.weight_decay': "Weight Decay",
    'in_params.train_params.initial_lr': 'LR',
    'in_params.model.name': 'Model',
    'in_params.model.params.backbone': 'Backbone',
    'in_params.train_params.freeze_pretrained': 'Strategy',
    'in_params.model.params.main_pretrained': 'Used Weights',
    'train_WrapF1': 'Train F1',
    'f1': 'F1',
    'in_params.dataset.batch_size': 'Batch Size',
    'precision': 'Precision',
    'in_params.train_params.loss.params.gamma': 'Gamma',
    'in_params.model.params.fusion_type': 'DropPath',
    'recall': 'Recall'
}
value_renamings = {
    'Strategy': {True: 'Freeze', False: 'Fine-Tuning'},
    'DropPath': {'conv_sum_drop': 'Yes', 'conv_sum': 'No'}
}
channels_ordering = {
    "G, NDVI, NIR, R, RE" : 1,
    "R, G, NIR, RE": 2,
    "R, G, RE, NIR": 2,
    "R, G, RE": 3,
    "G, NIR, R": 3,
    "R, G, NIR": 4,
    "R, G, NDVI": 5,
    "R, G": 6,
    "CIR": 7,
    "NDVI": 8,

    'Fine-Tuning': 1,
    'Freeze': 2,

    'R': 1,
    'G': 2,
    'RGB': 2.5,
    'G, G, R': 3,
    'R, G, G, G': 4,

    'MiT-B0': 0,
    'MiT-B1': 1,
    'lawin': 0,
    'laweed': 1,
    'doublelawin': 2,
    'doublelaweed': 3,
}

## Lawin

In [34]:
filters = {
    'group': 'Lawin', 'config.in_params.model.params.backbone': 'MiT-B0'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    # 'in_params.model.name',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [35]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyper_params.dataset_params.return_path',
 'hyper_params.dataset_params.test_folders',
 'hyper_params.dataset_params.train_folders',
 'hyper_params.dataset_params.val_batch_size',
 

In [36]:
runs_df

,in_params.tags,in_params.model.name,in_params.model.params.backbone,in_params.phases,in_params.dataset.root,in_params.dataset.size,in_params.dataset.channels,in_params.dataset.hor_flip,in_params.dataset.ver_flip,in_params.dataset.crop_size,...,007_frame0224.png.masks.ground_truth.size,007_frame0224.png.masks.ground_truth._type,007_frame0224.png.masks.ground_truth.sha256,007_frame0224.png.width,007_frame0224.png.format,007_frame0224.png.height,007_frame0224.png.sha256,007_frame0224.png.path,007_frame0224.png.size,007_frame0224.png._type
0,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NDVI, NIR, R, RE]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,a6d5fb43ee3baf3d7f2ef0e06a1aece5bd258e21ba7a02...,media/images/007_frame0224.png_290_a6d5fb43ee3...,88038,image-file
1,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[R, G, RE, NIR]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,2137b194d2162e5303bf568673ec3118873ecad024d696...,media/images/007_frame0224.png_528_2137b194d21...,74508,image-file
2,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NIR, R]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,9a5d5ee3049b51823c6d0af5050c8aab0b7e00cc591c91...,media/images/007_frame0224.png_766_9a5d5ee3049...,87554,image-file
3,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,CIR,True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,8c2f5fe9bed274427150af87cbf48f8f1fe9742446d140...,media/images/007_frame0224.png_584_8c2f5fe9bed...,84858,image-file
4,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,[NDVI],True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,28b17ef56e139825bcae61e46abd75146ecd879c06ada7...,media/images/007_frame0224.png_528_28b17ef56e1...,38943,image-file
5,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[R, G]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,cb6c162ec21fc98edfa0d1f9f5fd014f9d9577c58350d0...,media/images/007_frame0224.png_584_cb6c162ec21...,73829,image-file
6,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NDVI, NIR, R, RE]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,e456d7936d57391fa660c4f09de40f3774ba566daf4bce...,media/images/007_frame0224.png_682_e456d7936d5...,64605,image-file
7,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[R, G, RE, NIR]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,6c1064c8c3fd1789174988abcd4f161f3fde4f526a87cf...,media/images/007_frame0224.png_668_6c1064c8c3f...,88112,image-file
8,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NIR, R]",True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,d7db98a58cc99bfa04872df5de61901d5090b22a2c0abf...,media/images/007_frame0224.png_682_d7db98a58cc...,61664,image-file
9,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,CIR,True,True,"[256, 256]",...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,8c2f5fe9bed274427150af87cbf48f8f1fe9742446d140...,media/images/007_frame0224.png_584_8c2f5fe9bed...,84858,image-file


In [37]:
projected_runs = runs_df[
    selected_cols
]

In [38]:
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [39]:
selected_runs

,in_params.dataset.channels,in_params.train_params.optimizer_params.weight_decay,in_params.train_params.initial_lr,f1,precision,recall
0,"[G, NDVI, NIR, R, RE]",0.0005,0.0001,0.326794,0.345053,0.465314
1,"[R, G, RE, NIR]",0.0005,0.0001,0.377526,0.705026,0.357784
2,"[G, NIR, R]",0.0005,0.0001,0.357331,0.499817,0.348049
3,CIR,0.0005,0.0001,0.404106,0.428261,0.403309
4,[NDVI],0.0005,0.0001,0.331022,0.338566,0.456207
5,"[R, G]",0.0005,0.0001,0.356745,0.390026,0.434191
6,"[G, NDVI, NIR, R, RE]",0.0000,0.0001,0.338468,0.390821,0.496304
7,"[R, G, RE, NIR]",0.0000,0.0001,0.372336,0.573994,0.355982
8,"[G, NIR, R]",0.0000,0.0001,0.371983,0.444056,0.481801
9,CIR,0.0000,0.0001,0.369274,0.399371,0.358478


In [40]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\2582322293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\2582322293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs[roundings] = selected_runs[roundings].round(3)


In [41]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\1057191433.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)


In [42]:
selected_runs

,Channels,Weight Decay,LR,F1,Precision,Recall
0,"G, NDVI, NIR, R, RE",0.0005,0.0001,0.327,0.345,0.465
1,"R, G, RE, NIR",0.0005,0.0001,0.378,0.705,0.358
2,"G, NIR, R",0.0005,0.0001,0.357,0.500,0.348
3,CIR,0.0005,0.0001,0.404,0.428,0.403
4,NDVI,0.0005,0.0001,0.331,0.339,0.456
5,"R, G",0.0005,0.0001,0.357,0.390,0.434
6,"G, NDVI, NIR, R, RE",0.0000,0.0001,0.338,0.391,0.496
7,"R, G, RE, NIR",0.0000,0.0001,0.372,0.574,0.356
8,"G, NIR, R",0.0000,0.0001,0.372,0.444,0.482
9,CIR,0.0000,0.0001,0.369,0.399,0.358


In [43]:
best_runs

,Channels,Weight Decay,LR,F1,Precision,Recall
18,"G, NDVI, NIR, R, RE",0.0000,0.0010,0.340,0.370,0.506
19,"R, G, RE, NIR",0.0000,0.0010,0.483,0.484,0.576
14,"G, NIR, R",0.0005,0.0010,0.400,0.543,0.378
23,"R, G",0.0000,0.0010,0.437,0.414,0.623
3,CIR,0.0005,0.0001,0.404,0.428,0.403
4,NDVI,0.0005,0.0001,0.331,0.339,0.456
22,NDVI,0.0000,0.0010,0.331,0.335,0.409


In [44]:
best_runs.to_csv("../../exps/lawin.csv")

In [45]:
selected_runs.to_csv("../../exps/lawin-complete.csv")

## Lawin Aug

In [ ]:
filters = {
    'group': 'LawinAugPatience'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
best_runs.to_csv("../../exps/lawin_aug.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin_aug-complete.csv")

## Lawin Pretrained

In [ ]:
filters = {
    '$or': [{'group': 'LawinPretrained'}, {'group': 'LawinFrozen'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/lawin_pretrained.csv")
selected_runs.to_csv("../../exps/lawin_pretrained-complete.csv")

## Focal

In [ ]:
filters = {
    'group': 'Focal'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/focal.csv")
selected_runs.to_csv("../../exps/focal-complete.csv")

## SRR

In [ ]:
filters = {
    'group': 'RandomRotation'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/rotation.csv")
selected_runs.to_csv("../../exps/rotation-complete.csv")

## Double Encoder

In [ ]:
filters = {
    'group': 'DoubleLawin'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/doublelawin.csv")
selected_runs.to_csv("../../exps/doublelawin-complete.csv")

## Double Encoder Pre-Trained

In [49]:
filters = {
    'group': 'DoubleLawinPretrained'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['Train F1', 'F1', 'Precision', 'Recall']

In [41]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyper_params.dataset_params.return_path',
 '

In [68]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [69]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\1898580816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\1898580816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\1898580816.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [70]:
selected_runs

,Channels,Gamma,Train F1,F1,Precision,Recall
0,"R, G, NIR, RE",2,0.82694,0.42325,0.65734,0.38692
1,"R, G, RE",2,0.79732,0.47719,0.62841,0.46428
2,"R, G, NDVI",2,0.79944,0.49919,0.68519,0.45200
3,"R, G, NIR",2,0.79872,0.47249,0.62196,0.43518
4,"R, G, NIR, RE",0,0.72349,0.54724,0.59197,0.55625
5,"R, G, RE",0,0.72777,0.56554,0.55958,0.71598
6,"R, G, NDVI",0,0.75601,0.55531,0.48353,0.86702
7,"R, G, NIR",0,0.70393,0.47226,0.57277,0.52975
8,"R, G, NIR, RE",2,0.81546,0.45437,0.66062,0.41898
9,"R, G, RE",2,0.80911,0.47035,0.64726,0.43482


In [71]:
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2396384400.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2396384400.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


In [72]:
best_runs

,Channels,Gamma,Train F1,F1,Precision,Recall
4,"R, G, NIR, RE",0,0.723,0.547,0.592,0.556
5,"R, G, RE",0,0.728,0.566,0.560,0.716
15,"R, G, NIR",0,0.727,0.519,0.577,0.571
10,"R, G, NDVI",2,0.829,0.601,0.562,0.674


In [78]:
selected_runs.mean()

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\560937940.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  selected_runs.mean()


Gamma        1.000000
Train F1     0.766125
F1           0.505250
Precision    0.594313
Recall       0.565250
dtype: float64

In [77]:
selected_runs.groupby(by=['Gamma']).mean()

,Train F1,F1,Precision,Recall
Gamma,,,,
0,0.718625,0.5325,0.553625,0.671375
2,0.813625,0.4780,0.635000,0.459125


In [ ]:
selected_runs.to_csv("../../exps/doublelawinpretrained-complete.csv")
best_runs.to_csv("../../exps/doublelawinpretrained.csv")

## Double Encoder Pre-Trained Drop

In [97]:
filters = {
    'group': 'DLPretrainedDrop'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall', 'Train F1']

In [98]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.fusion_type',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyp

In [99]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [100]:
selected_runs.fillna("", inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\1566115742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna("", inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\1566115742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\1566115742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [104]:
selected_runs

,Channels,Backbone,Gamma,Train F1,F1,Precision,Recall
0,"R, G, NIR, RE",MiT-B1,2.0,0.772,0.406,0.658,0.375
1,"R, G, RE",MiT-B1,2.0,0.734,0.433,0.650,0.393
2,"R, G, NDVI",MiT-B1,2.0,0.778,0.580,0.644,0.545
3,"R, G, NIR",MiT-B1,2.0,0.756,0.482,0.631,0.435
4,"R, G, NIR, RE",MiT-B1,,0.637,0.559,0.578,0.660
5,"R, G, RE",MiT-B1,,0.620,0.541,0.521,0.728
6,"R, G, NDVI",MiT-B1,,0.674,0.595,0.526,0.895
7,"R, G, NIR",MiT-B1,,0.606,0.510,0.502,0.747
8,"R, G, NIR, RE",MiT-B0,2.0,0.746,0.430,0.651,0.391
9,"R, G, RE",MiT-B0,2.0,0.770,0.458,0.667,0.415


In [105]:
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2396384400.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2396384400.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


In [106]:
best_runs

,Channels,Backbone,Gamma,Train F1,F1,Precision,Recall
12,"R, G, NIR, RE",MiT-B0,,0.629,0.560,0.534,0.643
5,"R, G, RE",MiT-B1,,0.620,0.541,0.521,0.728
7,"R, G, NIR",MiT-B1,,0.606,0.510,0.502,0.747
6,"R, G, NDVI",MiT-B1,,0.674,0.595,0.526,0.895


In [109]:
selected_runs.groupby(by=['Gamma']).mean()

,Train F1,F1,Precision,Recall
Gamma,,,,
,0.63175,0.535625,0.528000,0.742625
2.0,0.75850,0.466625,0.639125,0.454750


In [108]:
selected_runs.mean()

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\560937940.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  selected_runs.mean()


Train F1     0.695125
F1           0.501125
Precision    0.583562
Recall       0.598688
dtype: float64

In [107]:
selected_runs.to_csv("../../exps/dlpretraineddrop-complete.csv")
best_runs.to_csv("../../exps/dlpretraineddrop.csv")

## RGB Weights

In [5]:
filters = {
    'group': 'RedEdge'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [6]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

In [7]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [8]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Strategy', 'Used Weights'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2093829475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.fillna(0, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2093829475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_24360\2093829475.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [9]:
selected_runs

,Channels,Backbone,Strategy,Gamma,Used Weights,F1,Precision,Recall
20,"R, G, NIR, RE",MiT-B0,Fine-Tuning,2,"R, G, G, G",0.447,0.711,0.433
22,"R, G, NIR, RE",MiT-B0,Fine-Tuning,0,"R, G, G, G",0.616,0.653,0.697
21,"R, G, NIR, RE",MiT-B0,Freeze,2,"R, G, G, G",0.499,0.612,0.452
23,"R, G, NIR, RE",MiT-B0,Freeze,0,"R, G, G, G",0.534,0.496,0.753
16,CIR,MiT-B0,Fine-Tuning,2,"G, G, R",0.525,0.640,0.510
17,CIR,MiT-B0,Fine-Tuning,0,"G, G, R",0.603,0.589,0.756
12,CIR,MiT-B0,Freeze,2,"G, G, R",0.527,0.629,0.481
13,CIR,MiT-B0,Freeze,0,"G, G, R",0.545,0.517,0.767
18,NDVI,MiT-B0,Fine-Tuning,2,G,0.572,0.607,0.634
19,NDVI,MiT-B0,Fine-Tuning,0,G,0.577,0.503,0.923


In [268]:
selected_runs.to_csv("../../exps/rgb_weights-complete.csv")

In [59]:
filters = {
    '$or': [{'group': 'RedEdge'}, {'group': 'RandomRotation'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
]
roundings = ['F1']

In [60]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

In [61]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'
selected_runs.drop(selected_runs[selected_runs['Channels'].isin([['R', 'G']])].index, inplace=True)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\1874336447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\1874336447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.drop(selected_runs[selected_runs['Channels'].isin([['R', 'G']])].index, inplace=True)


In [62]:
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Used Weights'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\3723124597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs[roundings] = selected_runs[roundings].round(5)


In [63]:
idx = selected_runs.groupby(by=['Channels', 'Strategy', 'Used Weights'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels', 'Strategy'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\3340784096.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by=['Channels', 'Strategy'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\3340784096.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


In [15]:
best_runs

,Channels,Backbone,Strategy,Used Weights,F1
22,"R, G, NIR, RE",MiT-B0,Fine-Tuning,"R, G, G, G",0.616
23,"R, G, NIR, RE",MiT-B0,Freeze,"R, G, G, G",0.534
30,CIR,MiT-B1,Fine-Tuning,RGB,0.650
6,CIR,MiT-B1,Fine-Tuning,"G, G, R",0.669
13,CIR,MiT-B0,Freeze,"G, G, R",0.545
27,CIR,MiT-B1,Freeze,RGB,0.602
37,NDVI,MiT-B0,Fine-Tuning,R,0.594
8,NDVI,MiT-B1,Fine-Tuning,G,0.645
28,NDVI,MiT-B1,Freeze,R,0.685
9,NDVI,MiT-B1,Freeze,G,0.656


In [321]:
compare_runs = best_runs.merge(best_runs, on=['Channels', 'Strategy'])
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == compare_runs['Used Weights_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G, G, R'].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G'].index, inplace=True)
compare_runs

,Channels,Backbone_x,Strategy,Used Weights_x,F1_x,Backbone_y,Used Weights_y,F1_y
3,CIR,MiT-B1,Fine-Tuning,RGB,0.650,MiT-B1,"G, G, R",0.669
8,CIR,MiT-B1,Freeze,RGB,0.602,MiT-B0,"G, G, R",0.545
11,NDVI,MiT-B0,Fine-Tuning,R,0.594,MiT-B1,G,0.645
15,NDVI,MiT-B1,Freeze,R,0.685,MiT-B1,G,0.656


In [322]:
complete = best_runs[best_runs['Channels'] == 'R, G, NIR, RE']
complete.rename(columns={'Gamma': 'Gamma_y', 'F1': 'F1_y', 'Used Weights': 'Used Weights_y', 'Backbone': 'Backbone_x'}, inplace=True)
compare_runs = compare_runs.append(complete)
compare_runs.fillna("", inplace=True)
compare_runs

C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\4110542001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete.rename(columns={'Gamma': 'Gamma_y', 'F1': 'F1_y', 'Used Weights': 'Used Weights_y', 'Backbone': 'Backbone_x'}, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\4110542001.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare_runs = compare_runs.append(complete)


,Channels,Backbone_x,Strategy,Used Weights_x,F1_x,Backbone_y,Used Weights_y,F1_y
3,CIR,MiT-B1,Fine-Tuning,RGB,0.65,MiT-B1,"G, G, R",0.669
8,CIR,MiT-B1,Freeze,RGB,0.602,MiT-B0,"G, G, R",0.545
11,NDVI,MiT-B0,Fine-Tuning,R,0.594,MiT-B1,G,0.645
15,NDVI,MiT-B1,Freeze,R,0.685,MiT-B1,G,0.656
22,"R, G, NIR, RE",MiT-B0,Fine-Tuning,,,,"R, G, G, G",0.616
23,"R, G, NIR, RE",MiT-B0,Freeze,,,,"R, G, G, G",0.534


In [324]:
compare_runs.to_csv("../../exps/rgb_weights.csv")

## Laweed

In [46]:
filters = {
    'group': 'Laweed'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [47]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

In [48]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [49]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\1084863829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\1084863829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [50]:
selected_runs

,Channels,Gamma,Strategy,Used Weights,F1,Precision,Recall
0,"R, G",2.0,Fine-Tuning,0,0.436,0.615,0.420
1,"R, G",2,Freeze,0,0.000,0.000,0.000
2,"R, G",0,Fine-Tuning,0,0.549,0.521,0.776
3,"R, G",0,Freeze,0,0.526,0.504,0.675
4,CIR,2.0,Fine-Tuning,"G, G, R",0.429,0.653,0.458
5,CIR,2.0,Freeze,"G, G, R",0.536,0.613,0.526
6,CIR,0,Fine-Tuning,"G, G, R",0.605,0.563,0.822
7,CIR,0,Freeze,"G, G, R",0.519,0.475,0.824
8,NDVI,2.0,Fine-Tuning,0,0.507,0.621,0.588
9,NDVI,2.0,Freeze,0,0.596,0.648,0.587


In [336]:
best_runs

,Channels,Gamma,Strategy,Used Weights,F1,Precision,Recall
2,"R, G, NIR, RE",0.0,Fine-Tuning,"R, G, G, G",0.603,0.593,0.774
6,CIR,0.0,Fine-Tuning,0,0.622,0.622,0.702
10,NDVI,0.0,Fine-Tuning,G,0.587,0.523,0.870
3,"R, G, NIR, RE",0.0,Freeze,"R, G, G, G",0.530,0.494,0.773
7,CIR,0.0,Freeze,0,0.592,0.532,0.817
9,NDVI,2.0,Freeze,G,0.567,0.703,0.508


In [51]:
selected_runs.to_csv("../../exps/laweed-complete.csv")

In [108]:
filters = {
    '$and': [
        {'$or': [{'group': 'Laweed'}, {'group': 'RandomRotation'}, {'group': 'RedEdge'}]},
        {'config.in_params.model.params.backbone': 'MiT-B0'}
    ]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'in_params.model.name',
    'f1',
]
roundings = ['F1']

In [109]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs
selected_runs

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

,in_params.dataset.channels,in_params.train_params.loss.params.gamma,in_params.train_params.freeze_pretrained,in_params.model.params.main_pretrained,in_params.model.name,f1
0,"[R, G]",2.0,False,None,laweed,0.435744
1,"[R, G]",2,True,None,laweed,NaN
2,"[R, G]",NaN,False,None,laweed,0.549430
3,"[R, G]",NaN,True,None,laweed,0.526002
4,CIR,2.0,False,"[G, G, R]",laweed,0.428629
5,CIR,2.0,True,"[G, G, R]",laweed,0.535704
6,CIR,NaN,False,"[G, G, R]",laweed,0.605486
7,CIR,NaN,True,"[G, G, R]",laweed,0.518539
8,[NDVI],2.0,False,None,laweed,0.507502
9,[NDVI],2.0,True,None,laweed,0.595838


In [110]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Channels'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\618385479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\618385479.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [111]:
idx = selected_runs.groupby(by=['Channels', 'Model'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
best_runs

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\3963619940.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_1588\3963619940.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


,Channels,Gamma,Strategy,Used Weights,Model,F1
34,"R, G, NIR, RE",0,Fine-Tuning,"R, G, G, G",lawin,0.616
47,"R, G",0,Freeze,NaN,lawin,0.535
36,CIR,2,Fine-Tuning,RGB,lawin,0.625
26,NDVI,2,Freeze,G,lawin,0.646
14,"R, G, NIR, RE",0,Fine-Tuning,"R, G, G, G",laweed,0.603
2,"R, G",0,Fine-Tuning,None,laweed,0.549
18,CIR,0,Fine-Tuning,RGB,laweed,0.622
9,NDVI,2.0,Freeze,None,laweed,0.596


In [112]:
compare_runs = best_runs.merge(best_runs, on=['Channels'])
compare_runs.drop(compare_runs[compare_runs['Model_x'] == compare_runs['Model_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Model_x'] == 'laweed'].index, inplace=True)
compare_runs

,Channels,Gamma_x,Strategy_x,Used Weights_x,Model_x,F1_x,Gamma_y,Strategy_y,Used Weights_y,Model_y,F1_y
1,"R, G, NIR, RE",0,Fine-Tuning,"R, G, G, G",lawin,0.616,0,Fine-Tuning,"R, G, G, G",laweed,0.603
5,"R, G",0,Freeze,NaN,lawin,0.535,0,Fine-Tuning,None,laweed,0.549
9,CIR,2,Fine-Tuning,RGB,lawin,0.625,0,Fine-Tuning,RGB,laweed,0.622
13,NDVI,2,Freeze,G,lawin,0.646,2.0,Freeze,None,laweed,0.596


In [113]:
compare_runs.drop(columns=['Model_x', 'Model_y'], inplace=True)
compare_runs

,Channels,Gamma_x,Strategy_x,Used Weights_x,F1_x,Gamma_y,Strategy_y,Used Weights_y,F1_y
1,"R, G, NIR, RE",0,Fine-Tuning,"R, G, G, G",0.616,0,Fine-Tuning,"R, G, G, G",0.603
5,"R, G",0,Freeze,NaN,0.535,0,Fine-Tuning,None,0.549
9,CIR,2,Fine-Tuning,RGB,0.625,0,Fine-Tuning,RGB,0.622
13,NDVI,2,Freeze,G,0.646,2.0,Freeze,None,0.596


In [102]:
compare_runs.to_csv("../../exps/laweed.csv")

## Double Laweed

In [6]:
filters = {
    '$or': [{'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [7]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.fusion_type',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyp

In [8]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [9]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\4234819445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\4234819445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs['DropPath'].fillna('conv_sum', inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [12]:
selected_runs

,Channels,Gamma,DropPath,F1,Precision,Recall
0,"R, G, RE",2.0,Yes,0.481,0.658,0.439
1,"R, G, NDVI",2.0,Yes,0.555,0.662,0.519
2,"R, G, NIR",2.0,Yes,0.459,0.615,0.429
3,"R, G, NIR, RE",2.0,Yes,0.417,0.668,0.382
4,"R, G, RE",0,Yes,0.524,0.551,0.726
5,"R, G, NDVI",0,Yes,0.591,0.537,0.901
6,"R, G, NIR",0,Yes,0.517,0.549,0.593
7,"R, G, NIR, RE",0,Yes,0.593,0.581,0.636
8,"R, G, RE",2.0,No,0.465,0.649,0.445
9,"R, G, NDVI",2.0,No,0.599,0.676,0.563


In [13]:
selected_runs.to_csv("../../exps/doublelaweed-complete.csv")

In [36]:
filters = {
    '$or': [{'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'},{'group': 'DLPretrainedDrop'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'in_params.model.name',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.fusion_type',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyp

In [37]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [38]:
selected_runs

,in_params.dataset.channels,in_params.train_params.loss.params.gamma,in_params.model.params.fusion_type,in_params.model.name,f1
0,"[R, G, NIR, RE]",2.0,conv_sum_drop,doublelawin,0.405533
1,"[R, G, RE]",2.0,conv_sum_drop,doublelawin,0.433094
2,"[R, G, NDVI]",2.0,conv_sum_drop,doublelawin,0.579506
3,"[R, G, NIR]",2.0,conv_sum_drop,doublelawin,0.481747
4,"[R, G, NIR, RE]",NaN,conv_sum_drop,doublelawin,0.558757
5,"[R, G, RE]",NaN,conv_sum_drop,doublelawin,0.540953
6,"[R, G, NDVI]",NaN,conv_sum_drop,doublelawin,0.594600
7,"[R, G, NIR]",NaN,conv_sum_drop,doublelawin,0.510344
8,"[R, G, RE]",2.0,conv_sum_drop,doublelaweed,0.481271
9,"[R, G, NDVI]",2.0,conv_sum_drop,doublelaweed,0.555112


In [39]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Channels'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)



C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\2955175917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\2955175917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs['DropPath'].fillna('conv_sum', inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [40]:
idx = selected_runs.groupby(by=['Channels', 'Model'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
best_runs

C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\3963619940.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_29512\3963619940.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


,Channels,Gamma,DropPath,Model,F1
28,"R, G, NIR, RE",0,No,doublelawin,0.560
5,"R, G, RE",0,Yes,doublelawin,0.541
7,"R, G, NIR",0,Yes,doublelawin,0.510
6,"R, G, NDVI",0,Yes,doublelawin,0.595
15,"R, G, NIR, RE",0,Yes,doublelaweed,0.593
20,"R, G, RE",0,No,doublelaweed,0.533
14,"R, G, NIR",0,Yes,doublelaweed,0.517
17,"R, G, NDVI",2.0,No,doublelaweed,0.599


In [41]:
compare_runs = best_runs.merge(best_runs, on=['Channels'])

In [42]:
compare_runs

,Channels,Gamma_x,DropPath_x,Model_x,F1_x,Gamma_y,DropPath_y,Model_y,F1_y
0,"R, G, NIR, RE",0,No,doublelawin,0.560,0,No,doublelawin,0.560
1,"R, G, NIR, RE",0,No,doublelawin,0.560,0,Yes,doublelaweed,0.593
2,"R, G, NIR, RE",0,Yes,doublelaweed,0.593,0,No,doublelawin,0.560
3,"R, G, NIR, RE",0,Yes,doublelaweed,0.593,0,Yes,doublelaweed,0.593
4,"R, G, RE",0,Yes,doublelawin,0.541,0,Yes,doublelawin,0.541
5,"R, G, RE",0,Yes,doublelawin,0.541,0,No,doublelaweed,0.533
6,"R, G, RE",0,No,doublelaweed,0.533,0,Yes,doublelawin,0.541
7,"R, G, RE",0,No,doublelaweed,0.533,0,No,doublelaweed,0.533
8,"R, G, NIR",0,Yes,doublelawin,0.510,0,Yes,doublelawin,0.510
9,"R, G, NIR",0,Yes,doublelawin,0.510,0,Yes,doublelaweed,0.517


In [47]:
compare_runs.drop(compare_runs[compare_runs['Model_x'] == compare_runs['Model_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Model_x'] == 'doublelaweed'].index, inplace=True)
compare_runs

,Channels,Gamma_x,DropPath_x,Model_x,F1_x,Gamma_y,DropPath_y,Model_y,F1_y
1,"R, G, NIR, RE",0,No,doublelawin,0.560,0,Yes,doublelaweed,0.593
5,"R, G, RE",0,Yes,doublelawin,0.541,0,No,doublelaweed,0.533
9,"R, G, NIR",0,Yes,doublelawin,0.510,0,Yes,doublelaweed,0.517
13,"R, G, NDVI",0,Yes,doublelawin,0.595,2.0,No,doublelaweed,0.599


In [48]:
compare_runs.to_csv("../../exps/doublelaweed.csv")

# Final results

In [5]:
filters = {
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyper_params.dataset_params.return_path',
 'hyper_params.dataset_params.test_folders',
 '

In [6]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [7]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Model', 'Backbone'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

C:\Users\PasqualeDeMarinis\AppData\Local\Temp\ipykernel_15836\1509431000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\PasqualeDeMarinis\AppData\Local\Temp\ipykernel_15836\1509431000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs[roundings] = selected_runs[roundings].round(5)
C:\Users\PasqualeDeMarinis\AppData\Local\Temp\ipykernel_15836\1509431000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [8]:
best_runs

,Model,Backbone,F1
34,doublelawin,MiT-B1,0.595
55,splitlawin,MiT-B1,0.663
89,splitlawin,MiT-B0,0.597
96,doublelawin,MiT-B0,0.609
121,splitlaweed,MiT-B0,0.638
136,0,0,0.000
151,doublelaweed,MiT-B0,0.599
180,laweed,MiT-B0,0.622
188,lawin,MiT-B0,0.646
238,lawin,MiT-B1,0.685


In [9]:
filters = {
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyper_params.dataset_params.return_path',
 'hyper_params.dataset_params.test_folders',
 '

In [10]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs_channel = selected_runs[idx]
best_runs_channel[roundings] = best_runs_channel[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)
best_runs_channel

C:\Users\PasqualeDeMarinis\AppData\Local\Temp\ipykernel_15836\2452519722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\PasqualeDeMarinis\AppData\Local\Temp\ipykernel_15836\2452519722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs[roundings] = selected_runs[roundings].round(5)
C:\Users\PasqualeDeMarinis\AppData\Local\Temp\ipykernel_15836\2452519722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

,Channels,Model,Backbone,F1
18,"R, G",laweed,MiT-B0,0.549
52,"R, G, RE",splitlawin,MiT-B1,0.607
55,"R, G, NIR, RE",splitlawin,MiT-B1,0.663
58,CIR,lawin,MiT-B1,0.669
73,"NIR, G, R, G, NDVI, NIR, R, RE",segnet,0,0.338
96,"G, NDVI",doublelawin,MiT-B0,0.609
106,"R, G, NIR",splitlawin,MiT-B0,0.587
121,"R, G, NDVI",splitlaweed,MiT-B0,0.638
238,NDVI,lawin,MiT-B1,0.685
298,"G, NIR, R",lawin,MiT-B1,0.620
